In [1]:
import pandas as pd 
import pymysql
from datetime import datetime
from datetime import timedelta
import plotly.express as px
from RDS_connection import db, user, password
from mod_streamer_data import add_local_time, crop_timeframe, get_streams
from datetime import datetime
from dateutil import tz
import nltk
from nltk.tokenize import word_tokenize
import numpy as np

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\leesc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\leesc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
db = pymysql.connect(db, user, password)
cursor = db.cursor()

sql = '''use Streams'''
cursor.execute(sql)

query = '''
SELECT * FROM Streams
'''

df = pd.read_sql(query, db, index_col='ID')

#always run this after use
db.commit()
cursor.close()
db.close()

In [5]:
def count_freq(word, freq):
    for w in word:
        if w in list(freq.keys()):
            freq[w] += 1
        else:
            freq[w] = 1
    return freq

In [6]:
def add_local_time(streamer_data, cols):
    '''
    input: dataframe of streamer data, list of columns to be converted in utc time
    function: adds local timezone to dataframe as new column named f'{col}_toLocal'
    '''
    from datetime import datetime
    from dateutil import tz

    from_zone = tz.tzutc()
    to_zone = tz.tzlocal()

    #convert to local time
    for col in cols:
        converted_times = []
        for i in range(len(streamer_data)):
            utc = datetime.strptime(streamer_data[col].iloc[i], '%Y-%m-%d %H:%M:%S')
            utc = utc.replace(tzinfo=from_zone)
            central = utc.astimezone(to_zone)
            converted_times.append(central)
            
        streamer_data[f'{col}_toLocal'] = converted_times
    
    return streamer_data

In [14]:
name = 'scoped'
start_date = '2020-10-16 00:00:00'
end_date = '2020-11-1 00:00:00'

streamer_data = df[df['Name'] == name]

In [15]:
import string
from nltk.corpus import stopwords
table = str.maketrans(dict.fromkeys(string.punctuation))
freq = {}
for i in range(1,len(streamer_data)):
    if streamer_data['StreamTitle'].iloc[i-1] != streamer_data['StreamTitle'].iloc[i]:
        title = streamer_data['StreamTitle'].iloc[i]
        #remove tags
        title = title.split('!')[0]
        print(title)
        title = title.translate(table)
        title = title.lower()
        title = word_tokenize(title)
        title = [word for word in title if not word in stopwords.words()]
        
        freq = count_freq(title,freq=freq)
        
sorted_freq = {k: v for k, v in sorted(freq.items(), key=lambda item: item[1], reverse=True)}

pop = []
for key in sorted_freq.keys():
    if len(key) < 3:
        pop.append(key)
for key in pop:
    sorted_freq.pop(key)

Arena Trios | 30% off subs
Chipotle Tourney | 30% off subs 
Chipotle Tourney (west) | 30% off subs
Trio Scrims | 30% off subs
Pro Trio Scrims with Sentinels Highsky and A1 Tuexy | 30% off subs
Arab Trio Scrims with Sentinels Highsky and A1 Tuexy | 30% off subs
Pro Trio Scrims | 30% off subs
FNCS Warmup | 30% off subs
FNCS Semis | 30% off subs
High Kill Arena Solos | 30% off subs
Trio Arena w/ Co1azo & Tahi | 30% off subs
Arena Solos | 30% off subs
Arena Trios w/ Ceice & Blake | 30% off subs
Arena Solos | 30% off subs
Arena Solos into Scrims | 30% off subs
Warming up for trio scrims | 30% off subs
Pro Trio Scrims with Sen Highsky and Tuexy | 30% off subs
Pro Trio Scrims with Sentinels Highsky and A1 Tuexy | 30% off subs
Pro Trio Scrims with Sentinels Highsky and Tuexy | 30% off subs
Pro Trio Scrims with Sentinels Highsky and Tuexy | 30% off subs
2v2 Wagers with Tuexy 
2v2 Wagers with Tuexy 
2v2 ZW Wagers with Tuexy (CLOSED)
Pro Trio Scrims
Pro Trio Scrims with Sentinels Highsky and Tuex

In [16]:
top_n_keywords = 25

In [17]:
streamer_data

,Name,TimestampUTC,CurrentGameID,StreamTitle,ViewerCount,StarttimeUTC
ID,,,,,,
6829,scoped,2020-09-24 22:05:03,33214,Trio Chipotle Tourney | 30% off subs! !subtemb...,689,2020-09-24 22:00:55
7138,scoped,2020-09-24 22:10:03,33214,Trio Chipotle Tourney | 30% off subs! !subtemb...,2840,2020-09-24 22:00:55
7448,scoped,2020-09-24 22:15:03,33214,Trio Chipotle Tourney | 30% off subs! !subtemb...,4065,2020-09-24 22:00:55
7758,scoped,2020-09-24 22:20:03,33214,Trio Chipotle Tourney | 30% off subs! !subtemb...,4621,2020-09-24 22:00:55
8066,scoped,2020-09-24 22:25:03,33214,Trio Chipotle Tourney | 30% off subs! !subtemb...,5043,2020-09-24 22:00:55
...,...,...,...,...,...,...
2780003,scoped,2020-11-02 09:25:03,33214,$200 2v2 Bot Race,4643,2020-11-01 22:40:15
2780177,scoped,2020-11-02 09:30:03,33214,$200 2v2 Bot Race,4764,2020-11-01 22:40:15
2780352,scoped,2020-11-02 09:35:03,33214,$200 2v2 Bot Race,4909,2020-11-01 22:40:15


In [18]:
counts_per_word = {}
for key in list(sorted_freq.keys())[:top_n_keywords]:
    counts_per_word[key] = []
    for i in range(len(streamer_data)):
        if (streamer_data['StreamTitle'].iloc[i].split('!')[0].translate(table).lower().find(key) != -1): # if key is substring
            counts_per_word[key].append(streamer_data['ViewerCount'].iloc[i])
for key in counts_per_word.keys():
    counts_per_word[key] = np.mean(counts_per_word[key])
    
counts_per_word = {k: v for k, v in sorted(counts_per_word.items(), key=lambda item: item[1], reverse=True)}

In [19]:
counts_per_word

{'dreamhack': 6438.443037974684,
 'kill': 5652.767441860465,
 'pro': 5463.651810584958,
 'cash': 5394.939189189189,
 'sentinels': 5352.472727272727,
 'solo': 5346.892720306513,
 'subs': 5272.948484848484,
 'cup': 5258.3298429319375,
 'scrims': 5243.4301994301995,
 'trio': 5134.09842519685,
 'late': 5052.418181818181,
 'high': 5047.51,
 'night': 5037.9098360655735,
 'highsky': 5013.1294583883755,
 'arena': 4886.41469816273,
 'tuexy': 4736.462474645031,
 'inno': 4594.903225806452,
 'fncs': 4570.075,
 'solos': 4347.966666666666,
 'open': 4122.861842105263,
 '3v3': 4091.9655172413795,
 'co1azo': 4065.9646017699115,
 'wagers': 3891.8210116731516,
 'closed': 3876.8571428571427,
 '2v2': 3839.882716049383}

In [21]:
fig = px.bar(x=list(counts_per_word.keys()), y=list(counts_per_word.values()), labels={'x':'Category','y':'Avg_Viewership'},
             title=f'{name}\'s Average Viewership per Stream Title Keywords')
fig.show()